In [ ]:
import sys
#!{sys.executable} -m pip install matplotlib

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load

RANDOM_STATE = 7
random.seed(RANDOM_STATE)

In [ ]:
RANDOM_STATE

In [ ]:
def save_forest(ensemble, filename):
    dump(ensemble, filename  + ".joblib")
    
def generate_random_bitmask(n_bits, perc_1):
    attacker_bitmask = []
    for i in range(int(perc_1*n_bits)):
        attacker_bitmask.append(1)
    for i in range(n_trees - int(perc_1*n_bits)):
        attacker_bitmask.append(0)
    random.shuffle(attacker_bitmask)
    return attacker_bitmask
    
def extract_paths_from_bitmask(paths, attacker_bitmask, true_label):
    paths_to_convert = []
    for i, path_x_tree_list in enumerate(paths):
        path_to_covert_x_tree = []
        for path_in_tree in path_x_tree_list:
            if path_in_tree[-1][1] == true_label and attacker_bitmask[i] == 0:
                path_to_covert_x_tree.append(path_in_tree)
            if path_in_tree[-1][1] != true_label and attacker_bitmask[i] == 1:
                path_to_covert_x_tree.append(path_in_tree)
        paths_to_convert.append(path_to_covert_x_tree)
    return paths_to_convert

def create_constraint_problem(paths_to_convert):
    trees_paths_to_conj = []
    list_elem = []
    for paths_x_tree in paths_to_convert:
        tree_paths_to_disj = []
        for path_in_tree in paths_x_tree:
            path = []
            for node in path_in_tree[:-1]:
                list_elem.append(node[0])
                if(node[1] == '<='):
                    constr = elem[node[0]] <= node[2] - 0.0001 
                else:
                    constr = elem[node[0]] > node[2] + 0.0001
                path.append(constr)
            tree_paths_to_disj.append(And(*path))
        trees_paths_to_conj.append(Or(*tree_paths_to_disj))
    constr = And(*trees_paths_to_conj)
    return constr, list_elem

def generate_instances(constraints, n_instances, list_elem, ensemble):
    list_status_problems = []
    list_values = []
    tot_time = 0
    for i in range(n_instances):
        s = Solver()
        s.add(constraints)
        res = s.check()
        list_status_problems.append(res)
        if res == z3.sat:
            #print("Sat {}".format(i), end = "  -  ")
            for k, v in s.statistics():
                if(k == 'time'):
                    #print("%s : %s" % (k, v))
                    tot_time += v
            m = s.model()
            list_internal = []
            sol_values_constr = []
            list_elem = list(set(list_elem))
            found_instance_from_z3 = []
            for i in range(len(list_elem)):
                sol_values_constr.append(elem[i] == m[elem[i]])
            #print(Not(And(*sol_values_constr)))
            constraints = And(constraints, Not(And(*sol_values_constr))) 
            for i in range(ensemble.n_features_in_):
                if m[elem[i]] == None:
                    list_internal.append(float(0))
                else:
                    frac = m[elem[i]].as_fraction()
                    list_internal.append(float(frac.numerator) / float(frac.denominator))
            list_values.append(list_internal)
        else:
            #print(res, end = ", ")
            break
        
    return list_status_problems, list_values, tot_time, res

def check_predictions_with_watermark(instances, labels, ensemble, watermark):
    for instance, label in zip(instances, labels):
        for i, tree in enumerate(ensemble):
            predicted_label = int(tree.predict(np.array(instance).reshape((1, len(instance))))[0])
            if (watermark[i] == 0 and predicted_label != label) or (watermark[i] == 1 and predicted_label == label):
                print("Broken prediction on tree {} with class {} from tree, true label {} and watermark bit {}".format(i, predicted_label, label, watermark[i]))
                return False
    return True

def generate_extra_constraint(constr, eps, elem, test_instance):
    constr_add = []
    for i in range(len(elem)):
            constr_add.append(elem[i] >= test_instance[i] - eps)
            constr_add.append(elem[i] <= test_instance[i] + eps)
            constr_add.append(elem[i] >= 0)
            constr_add.append(elem[i] <= 1)
    constr = And(constr, And(*constr_add))
    return constr

def generate_range_constraint(constr, eps, elem, test_instance):
    constr_add = []
    for i in range(len(elem)):
            constr_add.append(elem[i] >= 0)
            constr_add.append(elem[i] <= 1)
    constr = And(constr, And(*constr_add))
    return constr

def generate_random_instance():
    synt_instance = []
    for f in range(len(elem)):
        synt_instance.append(random.random()) 
    #random label
    synt_instance_label = random.randint(0,1)
    return synt_instance, synt_instance_label

def generate_bitmask(instance, instance_label): 
    attacker_bitmask = []
    for t in ensemble_rit:
        for pred in t.predict([instance]):
            if pred == instance_label:
                attacker_bitmask.append(0)
            else:
                attacker_bitmask.append(1)
    return attacker_bitmask

def compute_accuracies_watermarked_model(ensemble, X, y):
    arr_ = []
    for i, tree in enumerate(ensemble):
        arr_.append(tree.predict(X))
        
    ones_ = 0
    zeros_ = 0
    pred_ = []
    for j in range(len(arr_[0])):
        for i in range(len(arr_)):
            if(arr_[i][j] == 1):
                ones_ = ones_+1
            else:
                zeros_ = zeros_+1
        if(ones_ > zeros_):
            pred_.append(1)
        else:
            pred_.append(0)
        ones_ = 0
        zeros_ = 0
    ensemble_acc = accuracy_score(y_true = y,  y_pred = pred_)
    print("Watermarked Model Accuracy: {:.3f}".format(ensemble_acc))

def compute_accuracies_standard_model(label, list_values, X_test, y_test, ensemble):
    y_ = []
    for i in range(len(list_values)):
        y_.append(label) #label
        
    trigger_synt_acc = accuracy_score(y_true = y_, y_pred = rf_standard.predict(list_values))
    print("Trigger synth acc: ", trigger_synt_acc, end = "  -  ")
    compute_accuracies_watermarked_model(ensemble, list_values, y_)
    print("Test set: ", accuracy_score(y_true = y_test, y_pred = rf_standard.predict(X_test)), end = "  -  ")
    compute_accuracies_watermarked_model(ensemble, X_test, y_test)

    return trigger_synt_acc

def ensemble_SC(n_trees, watermark, ones, zeros, X_train, y_train, X_test, y_test, dim, max_depth, max_leaves, stats, standard_accuracy):

    perc = dim*len(X_train)
    random.seed(RANDOM_STATE)
    all_instances = []
    for x in X_train:
        all_instances.append(x)
    sample = random.sample(all_instances, int(perc))

    labels_switched = []
    for i, x_i in enumerate(X_train):
        labels_switched.append(y_train[i])
        for j, instance in enumerate(sample):
            if np.array_equal(x_i, instance):
                if(y_train[i] == 1): labels_switched[i] = 0
                else: labels_switched[i] = 1

    labels = []
    for instance in sample:
        for i, x_i in enumerate(X_train):
            if np.array_equal(x_i, instance):
                labels.append(y_train[i])

    labels_s = copy.deepcopy(labels)
    for i in range(len(labels_s)):
        if labels_s[i] == 1:
            labels_s[i] = 0
        else:
          labels_s[i] = 1

    pos = []
    for i, x_i in enumerate(X_train):
            for s in sample:
                if np.array_equal(x_i, s):
                    pos.append(i)

    peso = 2
    cond = 0
    weights = []
    for i, x_i in enumerate(X_train):
            weights.append(1)

    n_attempt = 0
    while cond != 1 and n_attempt < 500: 
        new_arr = [] 
        for i in pos:
            weights[i] = peso

        trigger_rf = RandomForestClassifier(n_estimators=ones, max_depth = max_depth, max_leaf_nodes =  max_leaves, random_state = RANDOM_STATE, bootstrap = False, n_jobs = 3)
        trigger_rf.fit(X_train, labels_switched, sample_weight = weights)

        tot = 0
        num = 0
        for i, s in enumerate(sample):
            for t in trigger_rf.estimators_:
                if t.predict((np.array([s,]))) != labels[i]:
                    num = num+1
            if num == len(trigger_rf.estimators_):
              tot = tot + 1
            num = 0

        print("Percentage of trigger set instances correctly misclassified: {:.3f}\n\n".format(tot/len(sample)))

        if (cond < tot/len(sample)):
            cond = tot/len(sample)
            trigger_estimators = []
            for t in trigger_rf.estimators_:
                trigger_estimators.append(t)
        peso = peso + 30
        n_attempt += 1
    
    peso = 2
    cond = 0
    weights = []
    for i, x_i in enumerate(X_train):
            weights.append(1)
    n_attempt = 0
    while cond != 1 and n_attempt < 500: 
        new_arr = [] 
        for i in pos:
            weights[i] = peso

        rf = RandomForestClassifier(n_estimators=zeros, max_depth = max_depth, n_jobs = 3, max_leaf_nodes = max_leaves, random_state = RANDOM_STATE, bootstrap = False)
        rf.fit(X_train, y_train, sample_weight = weights)
        #trigger_estimators = trigger_rf.estimators_

        tot = 0
        num = 0
        for i, s in enumerate(sample):
            for t in rf.estimators_:
                if t.predict((np.array([s,]))) == labels[i]:
                    num = num+1
            if num == len(rf.estimators_):
              tot = tot + 1
            num = 0

        if (cond < tot/len(sample)):
            cond = tot/len(sample)
            estimators = []
            for t in rf.estimators_:
                estimators.append(t)
        peso = peso + 30
        n_attempt += 1

    tot = 0
    for i, s in enumerate(sample):
        for t in  rf.estimators_:
            if t.predict((np.array([s,]))) == labels[i]:
                num = num+1
        if num == len( rf.estimators_):
            tot = tot + 1
        num = 0

    print("Percentage of trigger set instances correctly classified: {:.3f}\n\n".format(tot/len(sample)))

    ensemble = []
    i = 0
    j = 0

    for digit in watermark:
        if digit == 1:
            ensemble.append(trigger_estimators[j])
            j = j+1
        else:
            ensemble.append(estimators[i])
            i = i+1
            
    arr = []
    for i, t in enumerate(ensemble):
        arr.append(t.predict(X_test))
        print("Label: {}  -  ".format(watermark[i]))
        print("Number of leaves: ", end="")
        print(t.get_n_leaves(), end = "  -  ")
        print("Depth: ", end="")
        print(t.get_depth())

    ones = 0
    zeros = 0
    pred = []
    for j in range(len(arr[0])):
        for i in range(len(arr)):
            if(arr[i][j] == 1):
                ones = ones+1
            else:
                zeros = zeros+1
        if(ones > zeros):
            pred.append(1)
        else:
            pred.append(0)
        ones = 0
        zeros = 0
    ensemble_acc = accuracy_score(y_true = y_test,  y_pred = pred)
    print("Ensemble Accuracy: {:.3f}".format(ensemble_acc))
    stats.append(ensemble_acc)
    print("Standard Model Accuracy: {:.3f}".format(standard_accuracy))
    print()
    return ensemble, sample, labels

def extract_paths_from_tree_aux(node_id, n_nodes, current_path, children_left, children_right, features, thresholds, values):
    if node_id >= n_nodes:
        return

    is_split_node = children_left[node_id] != children_right[node_id]
    paths = []

    if is_split_node:
        paths = extract_paths_from_tree_aux(children_left[node_id], n_nodes, current_path + [(features[node_id], "<=", thresholds[node_id])], children_left, children_right, features, thresholds, values)
        paths += extract_paths_from_tree_aux(children_right[node_id], n_nodes, current_path + [(features[node_id], ">", thresholds[node_id])], children_left, children_right, features, thresholds, values)
        return paths
    else:
        current_path.append(("-1", np.argmax(values[node_id])))
        return [current_path]

def extract_paths_from_tree(tree):

    n_nodes = tree.tree_.node_count
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    features = tree.tree_.feature
    thresholds = tree.tree_.threshold
    values = tree.tree_.value

    return extract_paths_from_tree_aux(0, n_nodes, [], children_left, children_right, features, thresholds, values)

In [ ]:
'''
f = open("../data/ijcnn1.tr", "r")
lines = f.readlines()
n_features = 22
data = np.zeros((len(lines), n_features+1))

for i, line in enumerate(lines):
	splitted_line = line.split(" ")
	data[i][0] = splitted_line[0]
	for values in splitted_line[1:]:
		v_splitted = values.split(":")
		if len(v_splitted) == 2:
			f, v = v_splitted
			data[i][int(f)] = v
		else:
			print("Values not parsed correctly! Line {}".format(i))
			print(v_splitted)

print("Saving data!")
np.savetxt("../data/icjnn1_train.csv", data, delimiter=",")
'''

In [ ]:
#Train set
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
data = pd.read_csv("../data/icjnn1_train.csv", delimiter=",", header=None)
data = resample(data, n_samples=10000, random_state=RANDOM_STATE, replace=False)
y_train = data.iloc[:, 0] = data.iloc[:, 0].astype(int).replace(-1, 0)
y_train = y_train.values
X_train = data.iloc[:, 1:] = data.iloc[:, 1:].astype(float).fillna(0)
X_train = X_train.values

X_train = np.nan_to_num(X_train)
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

#Test set
data = pd.read_csv("../data/icjnn1_test.csv", delimiter=",", header=None)
data = resample(data, n_samples=10000, random_state=RANDOM_STATE, replace=False)
y_test = data.iloc[:, 0] = data.iloc[:, 0].astype(int).replace(-1, 0)
y_test = y_test.values
X_test = data.iloc[:, 1:] = data.iloc[:, 1:].astype(float).fillna(0)
X_test = X_test.values

X_test = np.nan_to_num(X_test)
X_test = min_max_scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
rf_init = RandomForestClassifier(n_estimators = 90, random_state = RANDOM_STATE, bootstrap = False, n_jobs  = 3)
rf_init.fit(X_train,y_train)
estimators = rf_init.estimators_

max_leaves, max_depth, min_leaves, min_depth = 0, 0, 1000000, 1000000
for t in estimators:
    leaves = t.get_n_leaves()
    depth = t.get_depth()
    if max_leaves < leaves: max_leaves = leaves
    if max_depth < depth: max_depth = depth
    if min_leaves > leaves: min_leaves = leaves
    if min_depth > depth: min_depth = depth

print("Number of leaves: {} - {}".format(min_leaves, max_leaves))
print("Depth: {} - {}".format(min_depth, max_depth))

In [ ]:
n_estimators = list(range(50,101, 10))
max_depth = list(range(4,27, 2))
#max_leaf_nodes = list(range(80,101, 5))
bootstrap = [False]
random_state = [RANDOM_STATE]
n_jobs = [6]
param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              #'max_leaf_nodes': max_leaf_nodes,
              'bootstrap': bootstrap,
              'random_state': random_state,
              'n_jobs': n_jobs}

rf_grid = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf_grid, param_grid = param_grid, cv = 3, n_jobs = 6, verbose=4)
grid_search.fit(X_train, y_train)
best_grid = grid_search.best_estimator_

n_trees = 0
max_depth = 0
dictionare  = best_grid.get_params()
for i, param in enumerate(dictionare):
    if param == 'n_estimators':
        n_trees = dictionare['n_estimators']
    if param == 'max_depth':
        max_depth =  dictionare['max_depth']
print("Number of trees:{}".format(n_trees))
print("Depth: {}".format(max_depth))

In [ ]:
save_forest(best_grid, "../data/best_forest_ijcnn12")

In [ ]:
array_leaves = []
array_depth = []
max_leaves = 0
min_leaves = 100000
for t in best_grid.estimators_:
    leaves = t.get_n_leaves()
    array_leaves.append(leaves)
    if max_leaves < leaves: max_leaves = leaves
    if min_leaves > leaves: min_leaves = leaves

    array_depth.append(t.get_depth())

max_leaves = int(np.mean(array_leaves) - np.std(array_leaves))
print(max_leaves)

In [ ]:
np.std(array_leaves)

In [ ]:
watermark = []
for i in range(int(0.5*n_trees)):
    watermark.append(1)
for i in range(n_trees - int(0.5*n_trees)):
    watermark.append(0)
random.shuffle(watermark)

watermark = np.array(watermark)
dim = 0.02

leaves_per_forest = [] 
ones = (watermark == 1).sum()
zeros = (watermark == 0).sum()
print("Watermark: ", watermark)
print("Size: {}".format(dim))
stats = []
standard_accuracy = accuracy_score(y_true = y_test, y_pred = best_grid.predict(X_test))

ensemble_rit = ensemble_SC(n_trees, watermark, ones, zeros, X_train, y_train, X_test, y_test, dim, max_depth, max_leaves, stats, standard_accuracy)
save_forest(ensemble_rit,"../data/best_wm_forest_ijcnn12")

In [ ]:
print(watermark)

In [ ]:
# compute standard deviation and means
num_leaves_bit_0 = []
num_leaves_bit_1 = []
depths_bit_0 = []
depths_bit_1 = []
for i, tree in enumerate(ensemble_rit):
    if watermark[i] == 1: 
        num_leaves_bit_1.append(tree.get_n_leaves())
        depths_bit_1.append(tree.get_depth())
    if watermark[i] == 0:
        num_leaves_bit_0.append(tree.get_n_leaves())
        depths_bit_0.append(tree.get_depth())

std_leaves_bit_1 = np.std(num_leaves_bit_1)
means_leaves_bit_1 = np.mean(num_leaves_bit_1)
std_leaves_bit_0 = np.std(num_leaves_bit_0)
means_leaves_bit_0 = np.mean(num_leaves_bit_0)
print("LEAVES")
print("Bit 1: Std {} - Mean {}".format(std_leaves_bit_1, means_leaves_bit_1))
print("Bit 0: Std {} - Mean {}".format(std_leaves_bit_0, means_leaves_bit_0))
print()

std_depth_bit_1 = np.std(depths_bit_1)
means_depth_bit_1 = np.mean(depths_bit_1)
std_depth_bit_0 = np.std(depths_bit_0)
means_depth_bit_0 = np.mean(depths_bit_0)
print("DEPTH")
print("Bit 1: Std {} - Mean {}".format(std_depth_bit_1, means_depth_bit_1))
print("Bit 0: Std {} - Mean {}".format(std_depth_bit_0, means_depth_bit_0))

In [ ]:
n_trees = len(depths_bit_0 + depths_bit_1)

In [ ]:
mean_depths = np.mean(depths_bit_0 + depths_bit_1)
std_depths = np.std(depths_bit_0 + depths_bit_1)
std_criteria_depths = std_depths

In [ ]:
mean_depths

In [ ]:
std_depths

In [ ]:
wrong_1_depths = sum(depths_bit_0 > mean_depths + std_criteria_depths)
correct_0_depths = sum(depths_bit_0 < mean_depths - std_criteria_depths)
correct_1_depths = sum(depths_bit_1 > mean_depths + std_criteria_depths)
wrong_0_depths = sum(depths_bit_1 < mean_depths - std_criteria_depths)
uncertain = n_trees - wrong_1_depths - wrong_0_depths - correct_0_depths - correct_1_depths

print("BIT 1 WRONG: ", wrong_1_depths)
print("BIT 0 CORRECT: ", correct_0_depths)
print("BIT 1 CORRECT: ", correct_1_depths)
print("BIT 0 WRONG: ", wrong_0_depths)
print("UNCERTAIN: ", uncertain)
print("CORRECT: ", correct_0_depths + correct_1_depths)
print("WRONG: ", wrong_0_depths + wrong_1_depths)

In [ ]:
wrong_1_depths = sum(depths_bit_0 > mean_depths)
correct_0_depths = sum(depths_bit_0 < mean_depths)
correct_1_depths = sum(depths_bit_1 > mean_depths)
wrong_0_depths = sum(depths_bit_1 < mean_depths)
uncertain = n_trees - wrong_1_depths - wrong_0_depths - correct_0_depths - correct_1_depths

print("BIT 1 WRONG: ", wrong_1_depths)
print("BIT 0 CORRECT: ", correct_0_depths)
print("BIT 1 CORRECT: ", correct_1_depths)
print("BIT 0 WRONG: ", wrong_0_depths)
print("UNCERTAIN: ", uncertain)
print("CORRECT: ", correct_0_depths + correct_1_depths)
print("WRONG: ", wrong_0_depths + wrong_1_depths)

In [ ]:
mean_num_leaves = np.mean(num_leaves_bit_0 + num_leaves_bit_1)
std_num_leaves = np.std(num_leaves_bit_0 + num_leaves_bit_1)
std_criteria_leaves = std_num_leaves

In [ ]:
print("DEPTH")
print("mean: ", mean_depths)
print("std: ", std_depths)

print("LEAVES")
print("mean: ", mean_num_leaves)
print("std: ", std_num_leaves)

In [ ]:
wrong_1_leaves = sum(num_leaves_bit_0 > mean_num_leaves + std_criteria_leaves)
correct_0_leaves = sum(num_leaves_bit_0 < mean_num_leaves - std_criteria_leaves)
correct_1_leaves = sum(num_leaves_bit_1 > mean_num_leaves + std_criteria_leaves)
wrong_0_leaves = sum(num_leaves_bit_1 < mean_num_leaves - std_criteria_leaves)
uncertain = n_trees - wrong_1_leaves - wrong_0_leaves - correct_0_leaves - correct_1_leaves

print("BIT 1 WRONG: ", wrong_1_leaves)
print("BIT 0 CORRECT: ", correct_0_leaves)
print("BIT 1 CORRECT: ", correct_1_leaves)
print("BIT 0 WRONG: ", wrong_0_leaves)
print("UNCERTAIN: ", uncertain)
print("CORRECT: ", correct_0_leaves + correct_1_leaves)
print("WRONG: ", wrong_0_leaves + wrong_1_leaves)

In [ ]:
wrong_1_leaves = sum(num_leaves_bit_0 > mean_num_leaves)
correct_0_leaves = sum(num_leaves_bit_0 < mean_num_leaves)
correct_1_leaves = sum(num_leaves_bit_1 > mean_num_leaves)
wrong_0_leaves = sum(num_leaves_bit_1 < mean_num_leaves)
uncertain = n_trees - wrong_1_leaves - wrong_0_leaves - correct_0_leaves - correct_1_leaves

print("BIT 1 WRONG: ", wrong_1_leaves)
print("BIT 0 CORRECT: ", correct_0_leaves)
print("BIT 1 CORRECT: ", correct_1_leaves)
print("BIT 0 WRONG: ", wrong_0_leaves)
print("UNCERTAIN: ", uncertain)
print("CORRECT: ", correct_0_leaves + correct_1_leaves)
print("WRONG: ", wrong_0_leaves + wrong_1_leaves)

In [ ]:
num_leaves = []
depths = []
for i, tree in enumerate(ensemble_rit):
    num_leaves.append(tree.get_n_leaves())
    depths.append(tree.get_depth())

data.to_csv("../results/stats_wm_model_ijcnn1.csv")
data = pd.DataFrame([watermark, depths, num_leaves], index=["Bits", "Depths", "Leaves"])

In [ ]:
max_leaves

In [ ]:
num_leaves

In [ ]:
depths

# 3.3.3 new

In [ ]:
from z3 import *

In [ ]:
best_rf = load("../data/best_forest_ijcnn12.joblib")
ensemble_rit = load("../data/best_wm_forest_ijcnn12.joblib")

In [ ]:
len(ensemble_rit)

In [ ]:
paths = []
for i in range(len(ensemble_rit)):
    paths.append(extract_paths_from_tree(ensemble_rit[i]))

In [ ]:
elem = []
for i in range(len(X_train[0,:])):
    elem.append(Real('x'+str(i)))

#print(elem)

In [ ]:
from sklearn.utils import resample

import time

In [ ]:
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
n_trees = len(ensemble_rit)
#int(0.2*len(X_train))
subset_test_set_available_to_attacker, subset_labels_test_set_available_to_attacker = resample(X_test, y_test, n_samples=int(0.02*len(X_train)), random_state=RANDOM_STATE, replace=False)
print("N instances trigger set: ", int(0.02*len(X_train)))
trigger_sets_x_bitmask = []
bitmasks = []
results_x_bitmasks = []
epsilon_list = [0.3]#list(np.arange(0.1, 1.1, 0.2))
time_list = []

for epsilon in epsilon_list:
    print("RUN FOR EPSILON: ", epsilon)
    trigger_set_x_bitmask_x_eps = []
    results_x_bitmask_x_eps = []
    bitmasks_x_eps = []
    time_x_eps = []
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)
    for i in range(5):
        print("RUN FOR BITMASK: ", i+1)
        trigger_set = []
        #take test set instance
        #generate random bitmask
        attacker_bitmask = generate_random_bitmask(n_trees, 0.5)
        print("Attacker bitmask: ", attacker_bitmask)
        #label 1
        paths_to_convert = extract_paths_from_bitmask(paths, attacker_bitmask, 1)
        constr, list_elem = create_constraint_problem(paths_to_convert)
        #label 0
        paths_to_convert_2 = extract_paths_from_bitmask(paths, attacker_bitmask, 0)
        constr_2, list_elem_2 = create_constraint_problem(paths_to_convert_2)
        
        trigger_set_x_bitmask = []
        results_x_bitmask = []
        start = time.time()
        for idx in range(len(subset_test_set_available_to_attacker)):
            print("RUN FOR INSTANCE: ", idx)
            synthetic_instance = subset_test_set_available_to_attacker[idx]
            true_label = subset_labels_test_set_available_to_attacker[idx]
    
            if(true_label == 1):
                updated_constr = generate_extra_constraint(constr, epsilon, elem, synthetic_instance)
                list_status_problems, list_values, tot_time, res = generate_instances(updated_constr, 1, list_elem, best_rf)
            if(true_label == 0):
                updated_constr2 = generate_extra_constraint(constr_2, epsilon, elem, synthetic_instance)
                list_status_problems, list_values, tot_time, res = generate_instances(updated_constr2, 1, list_elem_2, best_rf)
        
            results_x_bitmask.append(res)
            if(res != unsat):
                print("SAT", end = ", ")
                trigger_set_x_bitmask.append([true_label] + list_values[0])
                print("Total time required to generate instances with true label {}: {}s".format(true_label, tot_time))
            
                check_instances_0 = check_predictions_with_watermark(list_values, [true_label]*len(list_values), ensemble_rit, attacker_bitmask)
                print("Check instances {}: {}".format(true_label, check_instances_0))
                if check_instances_0 == False:
                    for i, val in enumerate(list_values[0]):
                        print("{} -  {}".format(i, val))
                assert check_instances_0 == True
                print()

        end = time.time()
        time_x_eps.append(end-start)
        trigger_set_x_bitmask_x_eps.append(trigger_set_x_bitmask)
        bitmasks_x_eps.append(attacker_bitmask)
        results_x_bitmask_x_eps.append(results_x_bitmask)

    time_list.append(time_x_eps)
    results_x_bitmasks.append(results_x_bitmask_x_eps)
    trigger_sets_x_bitmask.append(trigger_set_x_bitmask_x_eps)
    bitmasks.append(bitmasks_x_eps)

In [ ]:
results_x_bitmasks

In [ ]:
sat_x_configs = []
for attempts_x_epsilon in results_x_bitmasks:
    sat_x_epsilon = []
    for attempts_x_bitmask in attempts_x_epsilon:
        sat_x_epsilon.append(sum(np.array(attempts_x_bitmask) == z3.sat))
    sat_x_configs.append(sat_x_epsilon)

In [ ]:
sat_x_configs

In [ ]:
sat_x_epsilon_x_bitmask = pd.DataFrame(sat_x_configs, index=list(np.arange(0.1, 1.1, 0.2)))
sat_x_epsilon_x_bitmask.to_csv("../results/report_sat_x_epsilon_x_bitmask_ijcnn1.csv", header=False)

In [ ]:
for ei, epsilon in enumerate(list(np.arange(0.1, 1.1, 0.2))):
    for bitmask_idx in range(5):
        pd.DataFrame(trigger_sets_x_bitmask[ei][bitmask_idx]).to_csv("../results/synth_trigger_set_ijcnn1_{}_{}.csv".format(str(epsilon)[:3], bitmask_idx), header=None, index=None)

In [ ]:
sat_x_epsilon_x_bitmask

In [ ]:
len(trigger_sets_x_bitmask[1][0][0])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)

pixels = np.array(trigger_sets_x_bitmask[1][0][1][1:]).reshape((28, 28))
ax[0].imshow(pixels, cmap='gray')

pixels = np.array(trigger_sets_x_bitmask[4][0][1][1:]).reshape((28, 28))
ax[1].imshow(pixels, cmap='gray')

plt.savefig("../results/example_z3_instances_03_09.pdf", bbox_inches="tight")